In [11]:
import math, hashlib, random, sys
import matplotlib.pyplot as plt


def hash_function(value):
    hash_value = hashlib.sha256(str(value).encode('utf8')).hexdigest()
    return int(hash_value, 16)

def leftmost_1_bit_position(hash_value, p):
    bin_hash = bin(hash_value)[2:]  # Convert hash to binary
    return bin_hash.find('1', p) + 1  # Find position of first '1' after p bits

def process_element(element, registers, m):
    hash_value = hash_function(element)
    p = int(math.log2(m))
    register_index = hash_value & (m - 1)  # Get first p bits for register index
    remaining_hash = hash_value >> p  # Shift out the first p bits
    position = leftmost_1_bit_position(remaining_hash, p)
    registers[register_index] = max(registers[register_index], position)
def harmonic_mean(registers):
    sum_of_inverses = sum([2**-reg for reg in registers])
    return len(registers) / sum_of_inverses

def bias_correction(registers, raw_estimate, m):
    if raw_estimate <= 2.5 * m:
        V = registers.count(0)
        if V > 0:
            return m * math.log(m / V)  # Linear counting
    elif raw_estimate > (2**32) / 30:
        return -(2**32) * math.log(1 - raw_estimate / (2**32))  # Large range correction
    return raw_estimate  # No correction needed

def estimate_cardinality(registers, a, b):
    alpha_m = a / (1 + b / len(registers))  # Alpha value for bias correction
    raw_estimate = alpha_m * len(registers)**2 * harmonic_mean(registers)
    return bias_correction(registers, raw_estimate, len(registers))

def hyperloglog_estimate(dataset, m, n, a=0.7213, b=1.079):
    registers = [0] * m
    for element in dataset:
        process_element(element, registers, m)
    estimate =  int(round(estimate_cardinality(registers, a, b),0))
    error = abs(100*(estimate/n - 1))
    return estimate, error, registers


## MAIN

def hll(N,M):
    dataset = []
    for _ in range(0,N):
        dataset.append(random.randint(1,M))
    n = len(set(dataset))

    pL = []; errorL = []
    min_p = None; min_estimate = None; min_error = 9999999
    for p in range(1,4):
        m = 2**p
        estimate = None; error = 9999999
        for n_a in range(1,10):
            for n_b in range(1,10):
                a = 0.7213 * n_a/5; b = 1.079 * n_b/5
                estimateT, errorT, registers = hyperloglog_estimate(dataset, m, n, a, b)
                if (error>errorT):
                    error = errorT
                    estimate = estimateT
        pL.append(p); errorL.append(error)
        if (min_error>error):
            min_p = p
            min_error = error
            min_estimate = estimate
            
    print(f"#dataset={N} real_difNums={n} estimate_difNums={min_estimate} min(p)={min_p} min(error)={min_error:.2f}%")
    sz1 = sys.getsizeof(registers)
    sz2 = sys.getsizeof(dataset)
    print(f"#registers={2**min_p} size(registers)={sz1} size(dataset)={sz2} Perc={100*sz1/sz2:.2f}%")

#plt.plot(pL,errorL)


In [12]:
import random, sys
import redis

## MAIN

def redishpll(N,M):
    dataset = []
    for _ in range(0,N):
        dataset.append(random.randint(1,M))
    n = len(set(dataset))

    # Connect to Redis
    r = redis.Redis(host='localhost', port=6379, db=0, decode_responses=True)

    # Add all elements to Redis HLL
    hll_key = "test_hll"
    r.delete(hll_key)  # Clean up
    for element in dataset:
        r.pfadd(hll_key, element)

    # Get estimate from Redis HLL
    estimate = r.pfcount(hll_key)
    error = abs(100*(estimate/n - 1))

    print(f"#dataset={N} real_difNums={n} estimate_difNums={estimate} error={error:.2f}%")

    # Get memory size
    info = r.memory_usage(hll_key)
    sz1 = info if info else 16  # HLL minimum size
    sz2 = sys.getsizeof(dataset)
    print(f"#size(hll)={sz1} size(dataset)={sz2} Perc={100*sz1/sz2:.2f}%")

In [ ]:
N = [1000, 10000, 100000, 1000000]
M=30

for n in N:
    redishpll(n, 30)
    hll(n, 30)
    print("----")

N= 1000
M= [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
for m in M:
    hll(N, m)
    redishpll(N, m)
    print("----")




#dataset=1000 real_difNums=30 estimate_difNums=30 error=0.00%
#size(hll)=248 size(dataset)=8856 Perc=2.80%
#dataset=1000 real_difNums=30 estimate_difNums=30 min(p)=1 min(error)=0.00%
#registers=2 size(registers)=120 size(dataset)=8856 Perc=1.36%
----
#dataset=10000 real_difNums=30 estimate_difNums=30 error=0.00%
#size(hll)=248 size(dataset)=85176 Perc=0.29%
#dataset=10000 real_difNums=30 estimate_difNums=30 min(p)=1 min(error)=0.00%
#registers=2 size(registers)=120 size(dataset)=85176 Perc=0.14%
----
#dataset=100000 real_difNums=30 estimate_difNums=30 error=0.00%
#size(hll)=248 size(dataset)=800984 Perc=0.03%
#dataset=100000 real_difNums=30 estimate_difNums=30 min(p)=1 min(error)=0.00%
#registers=2 size(registers)=120 size(dataset)=800984 Perc=0.01%
----
#dataset=1000000 real_difNums=30 estimate_difNums=30 error=0.00%
#size(hll)=248 size(dataset)=8448728 Perc=0.00%


In [ ]:
hll
#dataset=1000 real_difNums=30 estimate_difNums=30 min(p)=1 min(error)=0.00%
#registers=2 size(registers)=120 size(dataset)=8856 Perc=1.36%
#dataset=1000 real_difNums=30 estimate_difNums=30 error=0.00%
#size(hll)=248 size(dataset)=8856 Perc=2.80%

redishll
#dataset=1000 real_difNums=30 estimate_difNums=30 error=0.00%
#size(hll)=168 size(dataset)=8856 Perc=1.90%
#dataset=1000 real_difNums=30 estimate_difNums=30 min(p)=1 min(error)=0.00%
#registers=2 size(registers)=120 size(dataset)=8856 Perc=1.36%